#Visual Language Modeling for Automated Large-Scale Archaeological Site Detection Across the Amazon: An End-to-End Workflow Inspired by the OpenAI to Z Challenge
Iban Berganzo-Besga (https://orcid.org/0000-0002-6161-2452), Hector A. Orengo (https://orcid.org/0000-0002-9385-2370)

*Barcelona Supercomputing Center (BSC-CNS), Barcelona 08034, Spain*


In [1]:
!pip install openai

In [2]:
!pip show openai

Name: openai
Version: 1.91.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [3]:
import openai

# Set your OpenAI API key
openai.api_key = ""

In [5]:
!git clone https://github.com/iberganzo/OpenAItoZChallenge.git

Cloning into 'OpenAItoZChallenge'...
remote: Enumerating objects: 369, done.
remote: Counting objects: 100% (369/369), done.
remote: Compressing objects: 100% (352/352), done.
remote: Total 369 (delta 86), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (369/369), 1.81 MiB | 6.17 MiB/s, done.
Resolving deltas: 100% (86/86), done.


##PART I: survey known sites
###o3 structured data extraction and further geoprocessing

In [3]:
!pip install PyMuPDF pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 83.3 MB/s eta 0:00:00


In [8]:
import fitz  # PyMuPDF
import requests
from PIL import Image
import openai
import base64
from io import BytesIO
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Article provided by OpenAI to Z Challenge
# https://www.nature.com/articles/s41467-018-03510-7.pdf
# Supplementary Information PDF
local_pdf = 0  # 0 = use URLs, 1 = use local files
pdf_path = "supplementary.pdf"
pdf_url = "https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-018-03510-7/MediaObjects/41467_2018_3510_MOESM1_ESM.pdf"

output_shapefile = "known_sites_detected.shp"
ocr_lang = "eng"  # Change to "spa" if PDF is in Spanish or "por" if is in Portuguese

model_name = "o3-2025-04-16"
max_pages = 10  # Limit number of pages to analyze (avoid too many API calls)

# Download PDF from URL and open with PyMuPDF
def load_pdf_from_url(url):
    response = requests.get(url)
    return fitz.open(stream=response.content, filetype="pdf")

# Render each page as PIL image
def render_pdf_to_images(doc, max_pages=None):
    images = []
    for i, page in enumerate(doc):
        if max_pages and i >= max_pages:
            break
        pix = page.get_pixmap(dpi=200)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append(img)
    return images

# Convert PIL image to base64 for OpenAI input
def encode_image_to_base64(pil_img):
    buffer = BytesIO()
    pil_img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode()

# Use OpenAI o3 to extract site data from image
def extract_sites_with_o3(pil_img):
    base64_img = encode_image_to_base64(pil_img)

    response = openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "Extract archaeological site data from the image. Each row should contain: name, type (if available), area (if available), latitude and longitude."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Extract structured data from this image (name, type, area, latitude, longitude):"},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_img}"}}
                ]
            }
        ]
    )

    return response.choices[0].message.content

# Parse the response from OpenAI (basic row extractor)
def parse_sites(text):
    sites = []
    for line in text.splitlines():
        # Skip empty lines
        if not line.strip():
            continue

        # Try to split by comma
        parts = [p.strip() for p in line.split(",")]
        if len(parts) < 5:
            continue  # not enough parts

        name = parts[0]
        type_ = parts[1] if parts[1] else None
        area = parts[2] if parts[2] else None

        try:
            lon = float(parts[3])
            lat = float(parts[4])
        except ValueError:
            continue  # invalid coordinates

        sites.append({
            "site": name,
            "type": type_,
            "area": area,
            "lat": lat,
            "lon": lon
        })
    return sites

# Save extracted sites to Shapefile
def save_sites_to_shapefile(sites, output_path):
    df = pd.DataFrame(sites)
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.lon, df.lat),
        crs="EPSG:4326"
    )
    gdf.to_file(output_path)
    print(f"Shapefile saved: {output_path}")

# Main
print("Downloading PDF...")
doc = load_pdf_from_url(pdf_url)

print("Rendering pages...")
images = render_pdf_to_images(doc, max_pages=max_pages)

all_sites = []

for idx, img in enumerate(images):
    print(f"\nProcessing page {idx+1}...")
    try:
        result = extract_sites_with_o3(img)
        print("Raw response:\n", result)
        parsed = parse_sites(result)
        print(f"✅ Sites extracted: {len(parsed)}")
        all_sites.extend(parsed)
    except Exception as e:
        print(f"Error on page {idx+1}: {e}")

if all_sites:
    save_sites_to_shapefile(all_sites, output_shapefile)
else:
    print("⚠️ No valid site data found.")

Rendering pages...

Processing page 1...
Raw response:
 No archaeological site entries (name, type, area, latitude, longitude) are present in the supplied image.
✅ Sites extracted: 0

Processing page 2...
Raw response:
 The page shown contains only general descriptive text (no specific site names or coordinates).  
No archaeological site data (name + type + area + latitude + longitude) can be extracted from this image.
✅ Sites extracted: 0

Processing page 3...
Raw response:
 No precise geographic coordinates are provided in the text, and the passages refer only to broad site clusters rather than individually named sites. The extract therefore yields very limited structured data:

Name                               | Type (if available)                              | Area (ha, if mentioned) | Latitude | Longitude
---------------------------------- | ------------------------------------------------ | ----------------------- | -------- | ----------
Baures ring-ditch sites (Bolivia)  | Fo

## PART II: site detection
###GPT-4.1 few-shot binary visual classification using enhanced vegetation index (EVI) satellite imagery

In [9]:
!mkdir images_features_class0
!mkdir images_features_class1
!mkdir context_examples
!mkdir context_examples/class0
!mkdir context_examples/class1

!mkdir detected_features_class0
!mkdir detected_features_class1

!mkdir detected_features

In [10]:
#EVI imagery#
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/context_examples/class0/*.jpg context_examples/class0
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/context_examples/class1/*.jpg context_examples/class1
#Validation dataset
#!cp /content/OpenAItoZChallenge/datasets/evi_imagery/images_features_class0/val/*.jpg images_features_class0/
#!cp /content/OpenAItoZChallenge/datasets/evi_imagery/images_features_class1/val/*.jpg images_features_class1/
#Test dataset
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/images_features_class0/test/*.jpg images_features_class0/
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/images_features_class1/test/*.jpg images_features_class1/

In [7]:
#!rm -r images_features_class0/*
#!rm -r images_features_class1/*

#!rm -r context_examples/class0/*
#!rm -r context_examples/class1/*

In [11]:
!ls images_features_class0 | wc -l
!ls images_features_class1 | wc -l

!ls context_examples/class0 | wc -l
!ls context_examples/class1 | wc -l

15
15
2
2


In [12]:
from PIL import Image, ImageDraw, ImageFont
import os
import base64
from io import BytesIO

# Performance metrics #
# Confidence Threshold = 0.5
# Validation dataset: F1 96.55% Recall 93.33% (14/15) Precision 100.00% (15/15)
# Confidence Threshold = 0.75
# Validation dataset: F1 80.00% Recall 66.67% (10/15) Precision 100.00% (15/15)
# Confidence threshold = 0.75 selected after analysing the missed detections by the threshold
# in the validation dataset. It is assumed that the Amazon will exhibit greater diversity than
# what is represented in this small but strict dataset; therefore, a high precision is prioritized,
# which is essential when surveying at large-scale. An F1 of 80% was enough in validation.
# Confidence Threshold = 0.75
# Test dataset: F1 75.00% Recall 60.00% (9/15) Precision 100.00% (15/15)

# Region names for 3x3 grid
region_names = [
    "NW", "N", "NE",
    "W",  "C", "E",
    "SW", "S", "SE"
]

confidence_threshold = 0.75

# Convert PIL image to base64
def pil_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Load few-shot context images
def get_context_images(context_folder):
    context_inputs = []
    for cls in ["class0", "class1"]:
        class_path = os.path.join(context_folder, cls)
        if not os.path.exists(class_path):
            continue

        for i, fname in enumerate(sorted(os.listdir(class_path))[:2]):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(class_path, fname)
                image = Image.open(path).convert("RGB")
                image_b64 = pil_to_base64(image)
                label = 0 if cls == "class0" else 1
                context_inputs.append({
                    "type": "text",
                    "text": f"Example image (class {label}):"
                })
                context_inputs.append({
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_b64}"
                    }
                })
    return context_inputs

# Classify an image using o4-mini
def classify_with_o4(image, context_folder="/content/context_examples"):
    image_b64 = pil_to_base64(image)
    context_images = get_context_images(context_folder)

    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in identifying archaeological sites in Enhanced Vegetation Index (EVI) satellite imagery. "
                "These sites often appear as concentrated, pseudo-circular green patches distinct from surrounding vegetation."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "Classify the following image strictly as one of these two classes:\n\n"
                        "Label 0: Contains a localized, pseudo-circular green patch likely indicating an archaeological site.\n"
                        "Label 1: Does not contain such a structure.\n\n"
                        "Reply only in the format:\n"
                        "Label: <0 or 1>\nConfidence: <float between 0 and 1>"
                        "Do not provide any other output. Choose the most likely label even if unsure."
                    )
                },
                *context_images,
                {
                    "type": "text",
                    "text": "Target image:"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_b64}"
                    }
                }
            ]
        }
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages,
        max_completion_tokens=50,
    )

    content = response.choices[0].message.content.strip()
    predicted_class = None
    confidence = 0.0
    try:
        for line in content.splitlines():
            if "Label" in line:
                predicted_class = int(line.split(":")[1].strip())
            if "Confidence" in line:
                confidence = float(line.split(":")[1].strip())
    except:
        predicted_class = None
        confidence = 0.0

    return content, predicted_class, confidence

# Find region with highest confidence for class 0
def locate_feature(image, context_folder="/content/context_examples"):
    width, height = image.size
    region_w, region_h = width // 3, height // 3

    best_region = "Unknown"
    best_conf = -1

    for row in range(3):
        for col in range(3):
            left = col * region_w
            upper = row * region_h
            right = left + region_w
            lower = upper + region_h

            region = image.crop((left, upper, right, lower))
            desc, cls, conf = classify_with_o4(region, context_folder=context_folder)

            if cls == 0 and conf > best_conf:
                best_conf = conf
                best_region = region_names[row * 3 + col]

    return best_region, best_conf if best_conf >= 0 else "N/A"

# Draw detected region box and confidence score
def draw_region_on_image(image, region_label, score):
    draw = ImageDraw.Draw(image)
    w, h = image.size
    region_w = w // 3
    region_h = h // 3

    region_map = {
        "NW": (0, 0), "N": (1, 0), "NE": (2, 0),
        "W":  (0, 1), "C": (1, 1), "E":  (2, 1),
        "SW": (0, 2), "S": (1, 2), "SE": (2, 2)
    }

    if region_label not in region_map:
        return image

    col, row = region_map[region_label]
    x0 = col * region_w
    y0 = row * region_h
    x1 = x0 + region_w
    y1 = y0 + region_h

    draw.rectangle([x0, y0, x1, y1], outline="red", width=3)

    score_text = f"{score:.2f}" if isinstance(score, float) else str(score)
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    try:
        bbox = font.getbbox(score_text)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]
    except:
        text_w, text_h = font.getsize(score_text)

    padding = 4
    box_x0 = x0 + (region_w - text_w) // 2 - padding
    box_y0 = y0 - text_h - 2 * padding
    box_x1 = box_x0 + text_w + 2 * padding
    box_y1 = box_y0 + text_h + 2 * padding

    if box_y0 < 0:
        box_y0 = y0
        box_y1 = y0 + text_h + 2 * padding

    draw.rectangle([box_x0, box_y0, box_x1, box_y1], fill="red")
    draw.text((box_x0 + padding, box_y0 + padding), score_text, fill="white", font=font)

    return image

# Main classification loop for a folder of images
def classify_images_in_folder(folder_path, detected_folder, output_txt="classification_results.txt", context_folder="/content/context_examples"):
    os.makedirs(detected_folder, exist_ok=True)

    count_feature = 0
    count_no_feature = 0
    results = []

    with open(output_txt, "w") as log_file:
        log_file.write("filename | predicted_label | likely region | confidence\n")

        for filename in os.listdir(folder_path):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(folder_path, filename)
                image = Image.open(path).convert("RGB")

                label_text, cls, conf = classify_with_o4(image, context_folder=context_folder)

                if cls is None:
                    cls = -1
                    region_str = "N/A"
                    confidence = "N/A"
                elif cls == 0 and conf >= confidence_threshold:
                    count_feature += 1
                    # region_str, bb_confidence = locate_feature(image, context_folder=context_folder)
                    # image_with_box = draw_region_on_image(image.copy(), region_str, bb_confidence)
                    #image_with_box.save(os.path.join(detected_folder, filename))
                    region_str = "N/A"
                    #image.save(os.path.join(detected_folder, filename))
                    confidence = conf
                else:
                    # Either class 1 or low confidence
                    count_no_feature += 1
                    region_str = "N/A"
                    confidence = conf

                log_line = f"{filename} | {cls} | {region_str} | {confidence}\n"
                print(log_line.strip())
                log_file.write(log_line)
                results.append((filename, cls, region_str, confidence))

    print("\n--- Results Summary ---")
    print(f"Feature class count (class 0): {count_feature}")
    print(f"No-feature class count (class 1): {count_no_feature}")
    print(f"Total: {count_feature + count_no_feature}")
    return results

# Main
if __name__ == "__main__":
    print("CLASS 0 (images with archaeological feature)")
    folder_0 = "/content/images_features_class0"
    detected_0 = "/content/detected_features_class0"
    classify_images_in_folder(folder_0, detected_0, context_folder="/content/context_examples")

    print("CLASS 1 (images without archaeological feature)")
    folder_1 = "/content/images_features_class1"
    detected_1 = "/content/detected_features_class1"
    classify_images_in_folder(folder_1, detected_1, context_folder="/content/context_examples")


CLASS 0 (images with archaeological feature)
work_dataset_evi_crop_5504_9600.jpg | 1 | N/A | 0.98
work_dataset_evi_crop_4352_9600.jpg | 0 | N/A | 0.87
work_dataset_evi_crop_3840_9088.jpg | 0 | N/A | 0.96
work_dataset_evi_crop_10112_5248.jpg | 0 | N/A | 0.75
work_dataset_evi_crop_3072_6016.jpg | 0 | N/A | 0.77
work_dataset_evi_crop_11776_4224.jpg | 1 | N/A | 0.93
work_dataset_evi_crop_12160_2432.jpg | 1 | N/A | 0.94
work_dataset_evi_crop_12928_2688.jpg | 0 | N/A | 0.75
work_dataset_evi_crop_9856_5504.jpg | 1 | N/A | 0.97
work_dataset_evi_crop_4096_6144.jpg | 0 | N/A | 0.87
work_dataset_evi_crop_4352_5504.jpg | 0 | N/A | 0.65
work_dataset_evi_crop_4224_9600.jpg | 0 | N/A | 0.81
work_dataset_evi_crop_5248_9472.jpg | 0 | N/A | 0.92
work_dataset_evi_crop_4864_5888.jpg | 0 | N/A | 0.82
work_dataset_evi_crop_10880_4736.jpg | 1 | N/A | 0.82

--- Results Summary ---
Feature class count (class 0): 9
No-feature class count (class 1): 6
Total: 15
CLASS 1 (images without archaeological feature)
wor

## PART III: detected sites validation
###GPT-4.1 few-shot binary visual classification using normalized difference red edge (NDRE) satellite imagery

In [13]:
!rm -r images_features_class0/*
!rm -r images_features_class1/*

!rm -r context_examples/class0/*
!rm -r context_examples/class1/*

In [14]:
#NDRE imagery#
!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/context_examples/class0/*.jpg context_examples/class0
!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/context_examples/class1/*.jpg context_examples/class1
#Validation dataset
#!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/images_features_class0/val/*.jpg images_features_class0/
#!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/images_features_class1/val/*.jpg images_features_class1/
#Test dataset
!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/images_features_class0/test/*.jpg images_features_class0/
!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/images_features_class1/test/*.jpg images_features_class1/

In [15]:
!ls images_features_class0 | wc -l
!ls images_features_class1 | wc -l

!ls context_examples/class0 | wc -l
!ls context_examples/class1 | wc -l

15
15
2
2


In [16]:
import openai
from PIL import Image, ImageDraw, ImageFont
import os
import base64
from io import BytesIO

# Performance metrics #
# Confidence Threshold = 0.5
# Validation dataset: F1 79.44% Recall 73.33% (11/15) Precision 86.67% (13/15)
# Confidence Threshold = 0.75
# Validation dataset: F1 77.78% Recall 66.67% (10/15) Precision 93.33% (14/15)
# Confidence threshold = 0.75 selected after analysing the missed detections by the threshold
# in the validation dataset. It is assumed that the Amazon will exhibit greater diversity than
# what is represented in this small but strict dataset; therefore, a high precision is prioritized,
# which is essential when surveying at large-scale. An F1 of 78% was enough in validation.
# Confidence Threshold = 0.75
# Test dataset: F1 88.89% Recall 80.00% (12/15) Precision 100.00% (15/15)

# Region names for 3x3 grid
region_names = [
    "NW", "N", "NE",
    "W",  "C", "E",
    "SW", "S", "SE"
]

confidence_threshold = 0.75

# Convert PIL image to base64
def pil_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Load few-shot context images
def get_context_images(context_folder):
    context_inputs = []
    for cls in ["class0", "class1"]:
        class_path = os.path.join(context_folder, cls)
        if not os.path.exists(class_path):
            continue

        for i, fname in enumerate(sorted(os.listdir(class_path))[:2]):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(class_path, fname)
                image = Image.open(path).convert("RGB")
                image_b64 = pil_to_base64(image)
                label = 0 if cls == "class0" else 1
                context_inputs.append({
                    "type": "text",
                    "text": f"Example image (class {label}):"
                })
                context_inputs.append({
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_b64}"
                    }
                })
    return context_inputs

# Classify an image using o4-mini
def classify_with_o4(image, context_folder="/content/context_examples"):
    image_b64 = pil_to_base64(image)
    context_images = get_context_images(context_folder)

    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in identifying rivers and paleochannels in Normalized Difference Red Edge (NDRE) satellite imagery. "
                "These features often appear as dark, continuous, and curvilinear structures, sometimes branching, crossing areas of vegetated or noisy background."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "Classify the following image strictly as one of these two classes:\n\n"
                        "Label 0: Contains a dark, continuous curvilinear structure resembling a river or paleochannel.\n"
                        "Label 1: No such structure is visible.\n\n"
                        "Reply only in the format:\n"
                        "Label: <0 or 1>\nConfidence: <float between 0 and 1>"
                        "Do not provide any other output. Choose the most likely label even if unsure."
                    )
                },
                *context_images,
                {
                    "type": "text",
                    "text": "Target image:"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_b64}"
                    }
                }
            ]
        }
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages,
        max_completion_tokens=50,
    )

    content = response.choices[0].message.content.strip()
    predicted_class = None
    confidence = 0.0
    try:
        for line in content.splitlines():
            if "Label" in line:
                predicted_class = int(line.split(":")[1].strip())
            if "Confidence" in line:
                confidence = float(line.split(":")[1].strip())
    except:
        predicted_class = None
        confidence = 0.0

    return content, predicted_class, confidence

# Find region with highest confidence for class 0
def locate_feature(image, context_folder="/content/context_examples"):
    width, height = image.size
    region_w, region_h = width // 3, height // 3

    best_region = "Unknown"
    best_conf = -1

    for row in range(3):
        for col in range(3):
            left = col * region_w
            upper = row * region_h
            right = left + region_w
            lower = upper + region_h

            region = image.crop((left, upper, right, lower))
            desc, cls, conf = classify_with_o4(region, context_folder=context_folder)

            if cls == 0 and conf > best_conf:
                best_conf = conf
                best_region = region_names[row * 3 + col]

    return best_region, best_conf if best_conf >= 0 else "N/A"

# Draw detected region box and confidence score
def draw_region_on_image(image, region_label, score):
    draw = ImageDraw.Draw(image)
    w, h = image.size
    region_w = w // 3
    region_h = h // 3

    region_map = {
        "NW": (0, 0), "N": (1, 0), "NE": (2, 0),
        "W":  (0, 1), "C": (1, 1), "E":  (2, 1),
        "SW": (0, 2), "S": (1, 2), "SE": (2, 2)
    }

    if region_label not in region_map:
        return image

    col, row = region_map[region_label]
    x0 = col * region_w
    y0 = row * region_h
    x1 = x0 + region_w
    y1 = y0 + region_h

    draw.rectangle([x0, y0, x1, y1], outline="red", width=3)

    score_text = f"{score:.2f}" if isinstance(score, float) else str(score)
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    try:
        bbox = font.getbbox(score_text)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]
    except:
        text_w, text_h = font.getsize(score_text)

    padding = 4
    box_x0 = x0 + (region_w - text_w) // 2 - padding
    box_y0 = y0 - text_h - 2 * padding
    box_x1 = box_x0 + text_w + 2 * padding
    box_y1 = box_y0 + text_h + 2 * padding

    if box_y0 < 0:
        box_y0 = y0
        box_y1 = y0 + text_h + 2 * padding

    draw.rectangle([box_x0, box_y0, box_x1, box_y1], fill="red")
    draw.text((box_x0 + padding, box_y0 + padding), score_text, fill="white", font=font)

    return image

# Main classification loop for a folder of images
def classify_images_in_folder(folder_path, detected_folder, output_txt="classification_results.txt", context_folder="/content/context_examples"):
    os.makedirs(detected_folder, exist_ok=True)

    count_feature = 0
    count_no_feature = 0
    results = []

    with open(output_txt, "w") as log_file:
        log_file.write("filename | predicted_label | likely region | confidence\n")

        for filename in os.listdir(folder_path):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(folder_path, filename)
                image = Image.open(path).convert("RGB")

                label_text, cls, conf = classify_with_o4(image, context_folder=context_folder)

                if cls is None:
                    cls = -1
                    region_str = "N/A"
                    confidence = "N/A"
                elif cls == 0 and conf >= confidence_threshold:
                    count_feature += 1
                    # region_str, bb_confidence = locate_feature(image, context_folder=context_folder)
                    # image_with_box = draw_region_on_image(image.copy(), region_str, bb_confidence)
                    #image_with_box.save(os.path.join(detected_folder, filename))
                    region_str = "N/A"
                    #image.save(os.path.join(detected_folder, filename))
                    confidence = conf
                else:
                    # Either class 1 or low confidence
                    count_no_feature += 1
                    region_str = "N/A"
                    confidence = conf

                log_line = f"{filename} | {cls} | {region_str} | {confidence}\n"
                print(log_line.strip())
                log_file.write(log_line)
                results.append((filename, cls, region_str, confidence))

    print("\n--- Results Summary ---")
    print(f"Feature class count (class 0): {count_feature}")
    print(f"No-feature class count (class 1): {count_no_feature}")
    print(f"Total: {count_feature + count_no_feature}")
    return results

# Main
if __name__ == "__main__":
    print("CLASS 0 (images with river feature)")
    folder_0 = "/content/images_features_class0"
    detected_0 = "/content/detected_features_class0"
    classify_images_in_folder(folder_0, detected_0, context_folder="/content/context_examples")

    print("CLASS 1 (images without river feature)")
    folder_1 = "/content/images_features_class1"
    detected_1 = "/content/detected_features_class1"
    classify_images_in_folder(folder_1, detected_1, context_folder="/content/context_examples")


CLASS 0 (images with river feature)
work_dataset_ndre_crop_3968_10624.jpg | 0 | N/A | 1.0
work_dataset_ndre_crop_11520_3456.jpg | 0 | N/A | 0.93
work_dataset_ndre_crop_12288_2432.jpg | 0 | N/A | 1.0
work_dataset_ndre_crop_9856_6912.jpg | 1 | N/A | 0.97
work_dataset_ndre_crop_4224_8960.jpg | 0 | N/A | 1.0
work_dataset_ndre_crop_11008_5248.jpg | 1 | N/A | 0.98
work_dataset_ndre_crop_4096_10112.jpg | 0 | N/A | 0.99
work_dataset_ndre_crop_4864_7168.jpg | 1 | N/A | 0.97
work_dataset_ndre_crop_4480_10368.jpg | 0 | N/A | 0.92
work_dataset_ndre_crop_12032_4096.jpg | 0 | N/A | 0.99
work_dataset_ndre_crop_3712_9088.jpg | 0 | N/A | 0.93
work_dataset_ndre_crop_2944_4352.jpg | 0 | N/A | 1.0
work_dataset_ndre_crop_5504_9472.jpg | 0 | N/A | 0.82
work_dataset_ndre_crop_2944_5888.jpg | 0 | N/A | 1.0
work_dataset_ndre_crop_10240_5248.jpg | 0 | N/A | 0.98

--- Results Summary ---
Feature class count (class 0): 12
No-feature class count (class 1): 3
Total: 15
CLASS 1 (images without river feature)
work_da

##PART IV: new survey
###potential sites detection (part II) and validation (part III) using GPT-4.1 and Sentinel-2 imagery (EVI and NDRE)

In [6]:
!apt-get update
!apt-get install megatools

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,067 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,040 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [7]:
!megadl https://mega.nz/#!oMEC2KTR!ytEXCXgy0MfItI2ogdka07r3xpbGgJ_FwZpQcu3IJuE

Downloaded survey_area.zip


In [8]:
# For new surveys: detection of new potential sites #
import zipfile
import os

# Add you own area to survey. The ZIP file contains the 128x128 EVI and NDRE images of the area
zip_path = '/content/survey_area.zip'
dest_folder = '/content/' # Destine folder

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dest_folder)

In [ ]:
!rm -r detected_features_class0/*
!rm -r detected_features_class1/*

!rm -r context_examples/class0/*
!rm -r context_examples/class1/*

In [22]:
#EVI imagery#
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/context_examples/class0/*.jpg context_examples/class0
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/context_examples/class1/*.jpg context_examples/class1

In [23]:
!ls survey_area/evi_imagery/jpg/ | wc -l

!ls context_examples/class0/ | wc -l
!ls context_examples/class1/ | wc -l

!ls detected_features_class0/ | wc -l

11554
2
2
0


In [13]:
import openai
from PIL import Image, ImageDraw, ImageFont
import os
import base64
from io import BytesIO
from natsort import natsorted

# Region names for 3x3 grid
region_names = [
    "NW", "N", "NE",
    "W",  "C", "E",
    "SW", "S", "SE"
]

confidence_threshold = 0.75 # restricted

# Convert PIL image to base64
def pil_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Load few-shot context images
def get_context_images(context_folder):
    context_inputs = []
    for cls in ["class0", "class1"]:
        class_path = os.path.join(context_folder, cls)
        if not os.path.exists(class_path):
            continue

        for i, fname in enumerate(sorted(os.listdir(class_path))[:2]):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(class_path, fname)
                image = Image.open(path).convert("RGB")
                image_b64 = pil_to_base64(image)
                label = 0 if cls == "class0" else 1
                context_inputs.append({
                    "type": "text",
                    "text": f"Example image (class {label}):"
                })
                context_inputs.append({
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_b64}"
                    }
                })
    return context_inputs

# Classify an image using gpt
def classify_with_o4(image, context_images):
    image_b64 = pil_to_base64(image)

    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in identifying archaeological sites in Enhanced Vegetation Index (EVI) satellite imagery. "
                "These sites often appear as concentrated, pseudo-circular green patches distinct from surrounding vegetation."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "Classify the following image strictly as one of these two classes:\n\n"
                        "Label 0: Contains a localized, pseudo-circular green patch likely indicating an archaeological site.\n"
                        "Label 1: Does not contain such a structure.\n\n"
                        "Reply only in the format:\n"
                        "Label: <0 or 1>\nConfidence: <float between 0 and 1>"
                        "Do not provide any other output. Choose the most likely label even if unsure."
                    )
                },
                *context_images,
                {
                    "type": "text",
                    "text": "Target image:"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_b64}"
                    }
                }
            ]
        }
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages,
        max_completion_tokens=50,
    )

    content = response.choices[0].message.content.strip()
    predicted_class = None
    confidence = 0.0
    try:
        for line in content.splitlines():
            if "Label" in line:
                predicted_class = int(line.split(":")[1].strip())
            if "Confidence" in line:
                confidence = float(line.split(":")[1].strip())
    except:
        predicted_class = None
        confidence = 0.0

    return content, predicted_class, confidence

# Find region with highest confidence for class 0
def locate_feature(image, context_folder="/content/context_examples"):
    width, height = image.size
    region_w, region_h = width // 3, height // 3

    best_region = "Unknown"
    best_conf = -1

    for row in range(3):
        for col in range(3):
            left = col * region_w
            upper = row * region_h
            right = left + region_w
            lower = upper + region_h

            region = image.crop((left, upper, right, lower))
            desc, cls, conf = classify_with_o4(region, context_folder=context_folder)

            if cls == 0 and conf > best_conf:
                best_conf = conf
                best_region = region_names[row * 3 + col]

    return best_region, best_conf if best_conf >= 0 else 0

# Draw detected region box and confidence score
def draw_region_on_image(image, region_label, score):
    draw = ImageDraw.Draw(image)
    w, h = image.size
    region_w = w // 3
    region_h = h // 3

    region_map = {
        "NW": (0, 0), "N": (1, 0), "NE": (2, 0),
        "W":  (0, 1), "C": (1, 1), "E":  (2, 1),
        "SW": (0, 2), "S": (1, 2), "SE": (2, 2)
    }

    if region_label not in region_map:
        return image

    col, row = region_map[region_label]
    x0 = col * region_w
    y0 = row * region_h
    x1 = x0 + region_w
    y1 = y0 + region_h

    draw.rectangle([x0, y0, x1, y1], outline="red", width=3)

    score_text = f"{score:.2f}" if isinstance(score, float) else str(score)
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    try:
        bbox = font.getbbox(score_text)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]
    except:
        text_w, text_h = font.getsize(score_text)

    padding = 4
    box_x0 = x0 + (region_w - text_w) // 2 - padding
    box_y0 = y0 - text_h - 2 * padding
    box_x1 = box_x0 + text_w + 2 * padding
    box_y1 = box_y0 + text_h + 2 * padding

    if box_y0 < 0:
        box_y0 = y0
        box_y1 = y0 + text_h + 2 * padding

    draw.rectangle([box_x0, box_y0, box_x1, box_y1], fill="red")
    draw.text((box_x0 + padding, box_y0 + padding), score_text, fill="white", font=font)

    return image

# Main classification loop for a folder of images
def classify_images_in_folder(folder_path, detected_folder, output_txt="classification_results.txt", context_folder="/content/context_examples"):
    os.makedirs(detected_folder, exist_ok=True)

    detected_sites_file = "detected_sites_without_river_validation.txt"
    detected_sites_log = open(detected_sites_file, "a")

    count_feature = 0
    count_no_feature = 0
    results = []

    context_images = get_context_images(context_folder)

    with open(output_txt, "w") as log_file:
        log_file.write("filename | predicted_label | likely region | confidence\n")

        num_images_analysed = 0
        for filename in natsorted(os.listdir(folder_path)):
            print(f"Image analysed: {filename} Num: {num_images_analysed}")
            num_images_analysed = num_images_analysed + 1
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(folder_path, filename)
                image = Image.open(path).convert("RGB")

                label_text, cls, conf = classify_with_o4(image, context_images)

                if cls is None:
                    cls = -1
                    region_str = "N/A"
                    confidence = "N/A"
                elif cls == 0 and conf >= confidence_threshold:
                    count_feature += 1
                    # region_str, bb_confidence = locate_feature(image, context_folder=context_folder)
                    # image_with_box = draw_region_on_image(image.copy(), region_str, bb_confidence)
                    #image_with_box.save(os.path.join(detected_folder, filename))
                    region_str = "N/A"
                    #image.save(os.path.join(detected_folder, filename))
                    detected_sites_log.write(filename + "\n")
                    detected_sites_log.flush()
                    confidence = conf
                else:
                    # Either class 1 or low confidence
                    count_no_feature += 1
                    region_str = "N/A"
                    confidence = conf

                log_line = f"{filename} | {cls} | {region_str} | {confidence}\n"
                if  cls == 0:
                  print(log_line.strip())
                  log_file.write(log_line)
                  results.append((filename, cls, region_str, confidence))
                else:
                  log_file.write(log_line)
                  results.append((filename, cls, region_str, confidence))

    print("\n--- Results Summary ---")
    print(f"Feature class count (class 0): {count_feature}")
    print(f"No-feature class count (class 1): {count_no_feature}")
    print(f"Total: {count_feature + count_no_feature}")

    detected_sites_log.close()

    return results

# Main
if __name__ == "__main__":
    print("Images with archaeological feature")
    image_folder = "/content/survey_area/evi_imagery/jpg/"
    detected_folder = "/content/detected_features_class0"
    classify_images_in_folder(image_folder, detected_folder, context_folder="/content/context_examples")


Se han truncado las últimas 5000 líneas del flujo de salida.
Image analysed: work_dataset_evi_crop_3072_12032.jpg Num: 2600
Image analysed: work_dataset_evi_crop_3072_12160.jpg Num: 2601
Image analysed: work_dataset_evi_crop_3072_12288.jpg Num: 2602
Image analysed: work_dataset_evi_crop_3072_12416.jpg Num: 2603
Image analysed: work_dataset_evi_crop_3072_12544.jpg Num: 2604
Image analysed: work_dataset_evi_crop_3072_12672.jpg Num: 2605
Image analysed: work_dataset_evi_crop_3072_12800.jpg Num: 2606
Image analysed: work_dataset_evi_crop_3072_12928.jpg Num: 2607
Image analysed: work_dataset_evi_crop_3072_13056.jpg Num: 2608
Image analysed: work_dataset_evi_crop_3072_13184.jpg Num: 2609
Image analysed: work_dataset_evi_crop_3072_13312.jpg Num: 2610
Image analysed: work_dataset_evi_crop_3072_13440.jpg Num: 2611
Image analysed: work_dataset_evi_crop_3072_13568.jpg Num: 2612
Image analysed: work_dataset_evi_crop_3072_13696.jpg Num: 2613
Image analysed: work_dataset_evi_crop_3072_13824.jpg Num:

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1 in organization org-jy2QRkkBG2mMwiXBAo2Xn4YK on tokens per min (TPM): Limit 30000, Used 26269, Requested 4002. Please try again in 542ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [14]:
import os
import shutil

def copy_listed_images(txt_file, src_folder, dst_folder):
    os.makedirs(dst_folder, exist_ok=True)

    with open(txt_file, "r") as f:
        filenames = [line.strip() for line in f.readlines() if line.strip()]

    for filename in filenames:
        src_path = os.path.join(src_folder, filename)
        dst_path = os.path.join(dst_folder, filename)

        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
            print(f"Copied: {filename}")
        else:
            print(f"File not found: {filename}")

# Main
txt_file = "detected_sites_without_river_validation.txt"
src_folder = "/content/survey_area/evi_imagery/jpg"
dst_folder = "/content/detected_features_class0"

copy_listed_images(txt_file, src_folder, dst_folder)

Copied: work_dataset_evi_crop_128_4224.jpg
Copied: work_dataset_evi_crop_128_7552.jpg
Copied: work_dataset_evi_crop_128_9600.jpg
Copied: work_dataset_evi_crop_128_9984.jpg
Copied: work_dataset_evi_crop_128_10368.jpg
Copied: work_dataset_evi_crop_256_2176.jpg
Copied: work_dataset_evi_crop_256_2304.jpg
Copied: work_dataset_evi_crop_256_4224.jpg
Copied: work_dataset_evi_crop_256_7552.jpg
Copied: work_dataset_evi_crop_256_9472.jpg
Copied: work_dataset_evi_crop_256_10112.jpg
Copied: work_dataset_evi_crop_256_12032.jpg
Copied: work_dataset_evi_crop_384_6016.jpg
Copied: work_dataset_evi_crop_384_6272.jpg
Copied: work_dataset_evi_crop_384_9088.jpg
Copied: work_dataset_evi_crop_384_13184.jpg
Copied: work_dataset_evi_crop_512_5248.jpg
Copied: work_dataset_evi_crop_512_7552.jpg
Copied: work_dataset_evi_crop_512_10368.jpg
Copied: work_dataset_evi_crop_512_13056.jpg
Copied: work_dataset_evi_crop_640_4608.jpg
Copied: work_dataset_evi_crop_640_4736.jpg
Copied: work_dataset_evi_crop_640_6784.jpg
Copie

In [15]:
!ls detected_features_class0 | wc -l

230


In [24]:
!rm -r context_examples/class0/*
!rm -r context_examples/class1/*

In [25]:
#NDRE imagery#
!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/context_examples/class0/*.jpg context_examples/class0
!cp /content/OpenAItoZChallenge/datasets/ndre_imagery/context_examples/class1/*.jpg context_examples/class1

In [27]:
!ls survey_area/ndre_imagery/jpg/ | wc -l

!ls context_examples/class0/ | wc -l
!ls context_examples/class1/ | wc -l

11554
2
2


In [20]:
import openai
from PIL import Image
import os
import base64
from io import BytesIO
import re
from natsort import natsorted

confidence_threshold = 0.75 #restricted

# Size of each image tile (in pixels)
tile_size = 128

# Convert a PIL image to base64 PNG format
def pil_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Load few-shot context examples from folder
def get_context_images(context_folder):
    context_inputs = []
    for cls in ["class0", "class1"]:
        class_path = os.path.join(context_folder, cls)
        if not os.path.exists(class_path):
            continue

        for i, fname in enumerate(sorted(os.listdir(class_path))[:2]):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(class_path, fname)
                image = Image.open(path).convert("RGB")
                image_b64 = pil_to_base64(image)
                label = 0 if cls == "class0" else 1
                context_inputs.append({"type": "text", "text": f"Example image (class {label}):"})
                context_inputs.append({
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{image_b64}"}
                })
    return context_inputs

# Classify image using o4-mini with a few-shot prompt
def classify_with_o4(image, context_images):
    image_b64 = pil_to_base64(image)

    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in identifying rivers and paleochannels in Normalized Difference Red Edge (NDRE) satellite imagery. "
                "These features often appear as dark, continuous, and curvilinear structures, sometimes branching, crossing areas of vegetated or noisy background."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "Classify the following image strictly as one of these two classes:\n\n"
                        "Label 0: Contains a dark, continuous curvilinear structure resembling a river or paleochannel.\n"
                        "Label 1: No such structure is visible.\n\n"
                        "Reply only in the format:\n"
                        "Label: <0 or 1>\nConfidence: <float between 0 and 1>"
                        "Do not provide any other output. Choose the most likely label even if unsure."
                    )
                },
                *context_images,
                {
                    "type": "text",
                    "text": "Target image:"
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{image_b64}"}
                }
            ]
        }
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages,
        max_completion_tokens=50,
    )

    content = response.choices[0].message.content.strip()
    predicted_class = None
    confidence = 0.0

    try:
        for line in content.splitlines():
            if "Label" in line:
                predicted_class = int(line.split(":")[1].strip())
            if "Confidence" in line:
                confidence = float(line.split(":")[1].strip())
    except:
        predicted_class = None
        confidence = 0.0

    return predicted_class, confidence

# Extract X, Y coordinates from filename
def parse_coords(filename):
    match = re.match(r"work_dataset_evi_crop_(\d+)_(\d+)\.jpg", filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None

# Generate 3x3 neighborhood NDRE filenames based on coordinates
def generate_ndre_neighbors(x, y, size):
    deltas = [(-size, -size), (0, -size), (size, -size),
              (-size, 0),     (0, 0),     (size, 0),
              (-size, size),  (0, size),  (size, size)]
    return [f"work_dataset_ndre_crop_{x+dx}_{y+dy}.jpg" for dx, dy in deltas]

# Classify whether a region or its neighbors contain a feature (class 0)
def classify_by_neighbors(evi_folder, ndre_folder, output_folder, context_folder="/content/context_examples"):
    os.makedirs(output_folder, exist_ok=True)

    detected_sites_file = "detected_sites_with_river_validation.txt"
    detected_sites_log = open(detected_sites_file, "a")

    context_images = get_context_images(context_folder)

    num_images_analysed = 0
    for evi_filename in natsorted(os.listdir(evi_folder)):
        print("Images analysed: ", num_images_analysed)
        num_images_analysed = num_images_analysed + 1
        if not evi_filename.startswith("work_dataset_evi_crop_") or not evi_filename.endswith(".jpg"):
            continue

        x, y = parse_coords(evi_filename)
        if x is None or y is None:
            continue

        neighbor_names = generate_ndre_neighbors(x, y, tile_size)
        found_feature = False

        for ndre_name in neighbor_names:
            ndre_path = os.path.join(ndre_folder, ndre_name)
            if not os.path.exists(ndre_path):
                continue

            ndre_img = Image.open(ndre_path).convert("RGB")
            pred_class, confidence = classify_with_o4(ndre_img, context_images)

            if pred_class == 0 and confidence >= confidence_threshold:
                found_feature = True
                break

        if found_feature:
            #src_path = os.path.join(evi_folder, evi_filename)
            #dst_path = os.path.join(output_folder, evi_filename)
            #Image.open(src_path).save(dst_path)
            detected_sites_log.write(evi_filename + "\n")
            detected_sites_log.flush()
            print(f"✅ Saved: {evi_filename} (positive detection): {confidence}")
        else:
            print(f"❌ Skipped: {evi_filename} (no detection)")

    detected_sites_log.close()

# Main
if __name__ == "__main__":
    classify_by_neighbors(
        evi_folder="/content/detected_features_class0",      # EVI image file names
        ndre_folder="/content/survey_area/ndre_imagery/jpg", # NDRE image tiles
        output_folder="/content/detected_features",          # Output folder for positive detections
        context_folder="/content/context_examples"           # Few-shot example images
    )


Images analysed:  0
✅ Saved: work_dataset_evi_crop_128_4224.jpg (positive detection): 0.97
Images analysed:  1
✅ Saved: work_dataset_evi_crop_128_7552.jpg (positive detection): 0.99
Images analysed:  2
✅ Saved: work_dataset_evi_crop_128_9600.jpg (positive detection): 0.84
Images analysed:  3
✅ Saved: work_dataset_evi_crop_128_9984.jpg (positive detection): 0.94
Images analysed:  4
✅ Saved: work_dataset_evi_crop_128_10368.jpg (positive detection): 0.83
Images analysed:  5
✅ Saved: work_dataset_evi_crop_256_2176.jpg (positive detection): 0.98
Images analysed:  6
✅ Saved: work_dataset_evi_crop_256_2304.jpg (positive detection): 0.98
Images analysed:  7
✅ Saved: work_dataset_evi_crop_256_4224.jpg (positive detection): 0.97
Images analysed:  8
✅ Saved: work_dataset_evi_crop_256_7552.jpg (positive detection): 1.0
Images analysed:  9
✅ Saved: work_dataset_evi_crop_256_9472.jpg (positive detection): 0.92
Images analysed:  10
❌ Skipped: work_dataset_evi_crop_256_10112.jpg (no detection)
Images 

In [22]:
import os
import shutil

def copy_listed_images(txt_file, src_folder, dst_folder):
    os.makedirs(dst_folder, exist_ok=True)

    with open(txt_file, "r") as f:
        filenames = [line.strip() for line in f.readlines() if line.strip()]

    for filename in filenames:
        src_path = os.path.join(src_folder, filename)
        dst_path = os.path.join(dst_folder, filename)

        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
            print(f"Copied: {filename}")
        else:
            print(f"File not found: {filename}")

# Main
txt_file = "detected_sites_with_river_validation.txt"
src_folder = "/content/survey_area/evi_imagery/jpg"
dst_folder = "/content/detected_features"

copy_listed_images(txt_file, src_folder, dst_folder)

Copied: work_dataset_evi_crop_128_4224.jpg
Copied: work_dataset_evi_crop_128_7552.jpg
Copied: work_dataset_evi_crop_128_9600.jpg
Copied: work_dataset_evi_crop_128_9984.jpg
Copied: work_dataset_evi_crop_128_10368.jpg
Copied: work_dataset_evi_crop_256_2176.jpg
Copied: work_dataset_evi_crop_256_2304.jpg
Copied: work_dataset_evi_crop_256_4224.jpg
Copied: work_dataset_evi_crop_256_7552.jpg
Copied: work_dataset_evi_crop_256_9472.jpg
Copied: work_dataset_evi_crop_384_6016.jpg
Copied: work_dataset_evi_crop_384_6272.jpg
Copied: work_dataset_evi_crop_384_9088.jpg
Copied: work_dataset_evi_crop_384_13184.jpg
Copied: work_dataset_evi_crop_512_5248.jpg
Copied: work_dataset_evi_crop_512_7552.jpg
Copied: work_dataset_evi_crop_512_13056.jpg
Copied: work_dataset_evi_crop_640_4736.jpg
Copied: work_dataset_evi_crop_640_7680.jpg
Copied: work_dataset_evi_crop_768_2048.jpg
Copied: work_dataset_evi_crop_768_3712.jpg
Copied: work_dataset_evi_crop_768_4480.jpg
Copied: work_dataset_evi_crop_768_4992.jpg
Copied: 

In [23]:
!ls detected_features | wc -l

200


In [24]:
!mkdir detected_features_bb

In [28]:
!rm -r context_examples/class0/*
!rm -r context_examples/class1/*

In [29]:
#EVI imagery#
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/context_examples/class0/*.jpg context_examples/class0
!cp /content/OpenAItoZChallenge/datasets/evi_imagery/context_examples/class1/*.jpg context_examples/class1

In [31]:
!ls survey_area/evi_imagery/jpg/ | wc -l

!ls context_examples/class0/ | wc -l
!ls context_examples/class1/ | wc -l

11554
2
2


In [32]:
import os
from PIL import Image, ImageDraw

region_names = [
    "NW", "N", "NE",
    "W",  "C", "E",
    "SW", "S", "SE"
]

region_map = {
    "NW": (0, 0), "N": (1, 0), "NE": (2, 0),
    "W":  (0, 1), "C": (1, 1), "E":  (2, 1),
    "SW": (0, 2), "S": (1, 2), "SE": (2, 2)
}

# Load few-shot context examples from folder
def get_context_images(context_folder):
    context_inputs = []
    for cls in ["class0", "class1"]:
        class_path = os.path.join(context_folder, cls)
        if not os.path.exists(class_path):
            continue

        for i, fname in enumerate(sorted(os.listdir(class_path))[:2]):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(class_path, fname)
                image = Image.open(path).convert("RGB")
                image_b64 = pil_to_base64(image)
                label = 0 if cls == "class0" else 1
                context_inputs.append({"type": "text", "text": f"Example image (class {label}):"})
                context_inputs.append({
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{image_b64}"}
                })
    return context_inputs

# Classify an image using gpt
def classify_with_o4(image, context_images):
    image_b64 = pil_to_base64(image)

    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in identifying archaeological sites in Enhanced Vegetation Index (EVI) satellite imagery. "
                "These sites often appear as concentrated, pseudo-circular green patches distinct from surrounding vegetation."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "Classify the following image strictly as one of these two classes:\n\n"
                        "Label 0: Contains a localized, pseudo-circular green patch likely indicating an archaeological site.\n"
                        "Label 1: Does not contain such a structure.\n\n"
                        "Reply only in the format:\n"
                        "Label: <0 or 1>\nConfidence: <float between 0 and 1>"
                        "Do not provide any other output. Choose the most likely label even if unsure."
                    )
                },
                *context_images,
                {
                    "type": "text",
                    "text": "Target image:"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_b64}"
                    }
                }
            ]
        }
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages,
        max_completion_tokens=50,
    )

    content = response.choices[0].message.content.strip()
    predicted_class = None
    confidence = 0.0
    try:
        for line in content.splitlines():
            if "Label" in line:
                predicted_class = int(line.split(":")[1].strip())
            if "Confidence" in line:
                confidence = float(line.split(":")[1].strip())
    except:
        predicted_class = None
        confidence = 0.0

    return content, predicted_class, confidence

def locate_feature(image, context_images):
    width, height = image.size
    region_w, region_h = width // 3, height // 3

    best_region = None
    best_conf = -1

    for row in range(3):
        for col in range(3):
            left = col * region_w
            upper = row * region_h
            right = left + region_w
            lower = upper + region_h

            region = image.crop((left, upper, right, lower))
            desc, cls, conf = classify_with_o4(region, context_images)

            if cls == 0 and conf > best_conf:
                best_conf = conf
                best_region = region_names[row * 3 + col]

    return best_region

def draw_region_on_image_no_confidence(image, region_label):
    draw = ImageDraw.Draw(image)
    w, h = image.size
    region_w = w // 3
    region_h = h // 3

    if region_label not in region_map:
        return image

    col, row = region_map[region_label]
    x0 = col * region_w
    y0 = row * region_h
    x1 = x0 + region_w
    y1 = y0 + region_h

    draw.rectangle([x0, y0, x1, y1], outline="red", width=3)
    return image


def classify_and_draw_all(folder_path, output_folder, output_txt_path="regions_detected.txt", context_folder="/content/context_examples"):
    os.makedirs(output_folder, exist_ok=True)

    context_images = get_context_images(context_folder)

    with open(output_txt_path, "w") as txt_file:
        for filename in sorted(os.listdir(folder_path)):
            if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            path = os.path.join(folder_path, filename)
            image = Image.open(path).convert("RGB")

            best_region = locate_feature(image, context_images)

            if best_region:
                image_with_box = draw_region_on_image_no_confidence(image.copy(), best_region)
            else:
                image_with_box = image

            image_with_box.save(os.path.join(output_folder, filename))

            # Name and region ("None" when no detected)
            region_str = best_region if best_region is not None else "None"
            txt_file.write(f"{filename};{region_str}\n")


# Main
if __name__ == "__main__":
    input_folder = "/content/detected_features"
    output_folder = "/content/detected_features_bb"
    output_txt = "/content/potential_sites_detected.txt"

    classify_and_draw_all(input_folder, output_folder, output_txt_path=output_txt)

In [33]:
import shutil
import os

# Folder to zip
zip_folder = "detected_features_bb"
zip_name = f"{zip_folder}.zip"

shutil.make_archive(zip_folder, 'zip', zip_folder)

if os.path.exists(zip_name):
    print(f"ZIP file: {zip_name}")
else:
    print("Error when creating ZIP file")

ZIP file: detected_features_bb.zip


*   Download the ZIP file detected_features_bb.zip
*   Download the TXT file potential_sites_detected.txt